## Problem C

A challenge in predictive analytics is to select a few good features out of many thousands of possibilities, so that a linear regression model estimated using these few features still has good predictive power. In this exercise, you will implement a reusable tool that solves this problem using mixed integer programming.

The input file is of Excel format and has one sheet. Each row represents an observation. The first column corresponds to the dependent variable the user wishes to predict. Each following column corresponds to a feature that may be used for prediction. The name of the dependent variable as well as the features are given as the first row.

![Sample input for 12.A](12-regression.png)

You should implements the following MIP formulation.

**Data:**

- $I$: the set of rows (`range(5)` in the example above, but should be inferred from the data).
- $J$: the set of features ("X1", "X2", $\cdots$, "X8" in the example above, but should be inferred from the data).
- $y_i$: the value of the independent variable in row $i$.
- $x_{ij}$: the value of feature $j$ in row $i$. 
- $k$: the maximum number of features used.
- $M$: a big constant.

**Decision Variables:**

- $\alpha$: the coefficient of the constant term. (Continuous)
- $\beta_j$: the coefficient for feature $j$. (Continuous)
- $p_i$: the predicted value for observation $i$. (Continuous)
- $z_j$: whether to use feature $j$. (Binary)

**Objective and Constraints:**

$$\begin{aligned}
\text{Minimize} && \sum_{i \in I} (y_i - p_i)^2 \\
\text{s.t} \\
\text{(Linear prediction)} && p_i &= \alpha + \sum_{j \in J}x_{ij} \beta_j && \text{for each row $i\in I$.} \\
\text{(Big M)} && -Mz_j \le \beta_j &\le Mz_j && \text{for each feature $j \in J$.} \\
\text{(Using few features)} && \sum_{j \in J}z_j &\le k
\end{aligned}$$

**Note that the variables $\alpha$, $\beta_j$ and $p_i$ are NOT constrained to be non-negative.**

**Write a function called "regress" to implement the above MIP with three input parameters: **

- inputFile: the path to an input file of the following format. 
- k: the value of the parameter $k$ (see list of data variables above).
- M: the value of the parameter $M$ (see list of data variables above).

**The function should return a pandas Series object with at most k+1 elements. The first entry has index "Constant" and the value corresponds to the optimal value of $\alpha$. For the subsequent elements, the index corresponds to the name of the features used, and the value correspond to the estimated coefficient $\beta_j$. Only the non-zero coefficients $\beta_j$ should be included.** See below for the Series returned using the input file above, and $k=2$, $M=100$.

In [46]:
import pandas as pd
from gurobi import GRB, Model

input = pd.read_excel('12-regression-input.xlsx')
k = 2
M = 100

I = range(len(input))
J = [i for i in input.columns if i != 'Y']

mod = Model()
a = mod.addVar(name = 'alpha',lb=-GRB.INFINITY)
b = mod.addVars(J,name = 'beta',lb=-GRB.INFINITY)
p = mod.addVars(I,name = 'p',lb=-GRB.INFINITY)
z = mod.addVars(J, name = 'z', vtype = GRB.BINARY)
mod.setObjective(sum((input.loc[i,'Y']-p[i])**2 for i in I), sense = GRB.MINIMIZE)
mod.addConstrs((p[i] == a + sum(input.loc[i,j]*b[j] for j in J) for i in I), name = 'p')
mod.addConstrs((-M*z[j] <= b[j] for j in J), name = 'Big M')
mod.addConstrs((M*z[j] >= b[j] for j in J), name = 'Big M')
mod.addConstr((sum(z[j] for j in J) <= k), name = 'Features')

# mod.update()
# mod.write('ProblemC2.lp')

# %cat 'ProblemC2.lp'

mod.setParam('OutputFlag',False)
mod.optimize()

ans = pd.Series(dtype = 'float64')
ans['Constant'] = a.x
for j in J:
    if z[j].x == 1:
        ans[j] = b[j].x
        
ans    

Constant    4.084086
X3          2.051074
X5          2.969166
dtype: float64

In [45]:
# Test code
coefficients = regress('12-regression-input.xlsx',2,100)
coefficients

Constant    4.084086
X3          2.051074
X5          2.969166
dtype: float64

**Write your function below. You must import all packages needed.**

In [44]:
# Final code
def regress(inputfile,k,M):
    
    import pandas as pd
    from gurobi import GRB, Model

    input = pd.read_excel(inputfile)

    I = range(len(input))
    J = [i for i in input.columns if i != 'Y']

    mod = Model()
    a = mod.addVar(name = 'alpha',lb=-GRB.INFINITY)
    b = mod.addVars(J,name = 'beta',lb=-GRB.INFINITY)
    p = mod.addVars(I,name = 'p',lb=-GRB.INFINITY)
    z = mod.addVars(J, name = 'z', vtype = GRB.BINARY)
    mod.setObjective(sum((input.loc[i,'Y']-p[i])**2 for i in I), sense = GRB.MINIMIZE)
    mod.addConstrs((p[i] == a + sum(input.loc[i,j]*b[j] for j in J) for i in I), name = 'p')
    mod.addConstrs((-M*z[j] <= b[j] for j in J), name = 'Big M')
    mod.addConstrs((M*z[j] >= b[j] for j in J), name = 'Big M')
    mod.addConstr((sum(z[j] for j in J) <= k), name = 'Features')

    # mod.update()
    # mod.write('ProblemC2.lp')

    # %cat 'ProblemC2.lp'

    mod.setParam('OutputFlag',False)
    mod.optimize()

    ans = pd.Series(dtype = 'float64')
    ans['Constant'] = a.x
    for j in J:
        if z[j].x == 1:
            ans[j] = b[j].x
            
    return ans    